In [1]:
import itertools

from mlscorecheck.aggregated import (
    Experiment,
    create_all_kfolds,
    fold_partitioning_generator,
    integer_partitioning_generator,
)
from mlscorecheck.check import check_1_dataset_known_folds_mos_scores

In [2]:
dataset = {'p': 10, 'n': 20}
folds = [{'p': 0, 'n': 5}, {'p': 5, 'n': 10}, {'p': 5, 'n': 5}]
evaluation = {'dataset': dataset,
                'folding': {'folds': folds},
                'aggregation': 'mos'}
experiment = {'evaluations': [evaluation],
                'aggregation': 'mos'}

experiment = Experiment(**experiment)

experiment.sample_figures(score_subset=['acc', 'spec'])

experiment.calculate_scores(score_subset=['acc', 'spec'])

evaluation ['acc', 'spec']
evaluation ['acc', 'spec']
evaluation ['acc', 'spec']


{'acc': 0.5888888888888889, 'spec': 0.6}

In [5]:
experiment.sample_figures(score_subset=['acc', 'spec'])

evaluation ['acc', 'spec']
evaluation ['acc', 'spec']


In [2]:
folding = {'folds': [{'p': 5, 'n': 10},
                        {'p': 0, 'n': 11}]}
eps = 1e-2
scores = {'acc': 0.95, 'spec': 0.97, 'sens': 0.93}

In [3]:
check_1_dataset_known_folds_mos_scores(dataset={'p': 5, 'n': 21},
                                        folding=folding,
                                        eps=eps,
                                        scores=scores)

aaa {'acc': 0.95, 'spec': 0.97, 'sens': 0.93}
fold ['acc', 'spec', 'sens']
aaa {'acc': 0.95, 'spec': 0.97, 'sens': 0.93}
fold ['acc', 'spec', 'sens']


ZeroDivisionError: float division by zero

In [9]:
list(fold_partitioning_generator(6, 6, 3, True, False))

[[(0, 3, 3), (4, 1, 1)], [(1, 2, 3), (3, 2, 1)], [(2, 2, 2), (2, 2, 2)]]

In [2]:
list(create_all_kfolds(6, 6, 3, True))

0
ps_a []
ps_b [0, 0, 6]
ps_b [0, 1, 5]
ps_b [0, 2, 4]
[0, 2, 4] [4, 2, 0]
1
ps_b [0, 3, 3]
[0, 3, 3] [4, 1, 1]
2
yielding [0, 3, 3] [4, 1, 1]
ps_b [1, 1, 4]
[1, 1, 4] [3, 3, 0]
2
yielding [1, 1, 4] [3, 3, 0]
ps_b [1, 2, 3]
[1, 2, 3] [3, 2, 1]
3
yielding [1, 2, 3] [3, 2, 1]
ps_b [2, 2, 2]
[2, 2, 2] [2, 2, 2]
3
yielding [2, 2, 2] [2, 2, 2]
1
2
3
4
5
6


[[(0, 3, 3), (1, 1, 4), (1, 2, 3), (2, 2, 2)],
 [(4, 1, 1), (3, 3, 0), (3, 2, 1), (2, 2, 2)]]

In [3]:
def all_integer_partitioning_generator(n, k):
    for m in range(1, min(k+1, n+1)):
        for ns in integer_partitioning_generator(n, m):
            yield [0] * (k - m) + ns

In [18]:
def check(ps, ns):
    ps_or_ns_zero = 0
    for idx in range(len(ps)):
        if ps[idx] < 0:
            return False
        if ns[idx] < 0:
            return False
        if ps[idx] == 0 or ns[idx] == 0:
            ps_or_ns_zero += 1
    if len(ps) - ps_or_ns_zero < 2:
        return False
    return True

In [34]:
def all_kfolds_generator(p, n, k):
    k_div = (p + n) // k
    k_mod = (p + n) % k

    print(k_div, k_mod)

    total_left = k_mod * (k_div + 1)
    total_right = (k - k_mod) * k_div

    for p_left in range(p+1):
        p_right = p - p_left
        n_left = total_left - p_left
        n_right = total_right - p_right

        if n_left < 0 or n_left > n or n_right < 0 or n_right > n:
            continue
        if p_left < 0 or p_left > p or p_right < 0 or p_right > p:
            continue

        print('p_left', p_left, p_right, n_left, n_right, total_left, total_right)

        if p_left == 0:
            print('aaa')
            ps_left = [0] * k_mod
            ns_left = [k_div + 1] * k_mod
            for ps in all_integer_partitioning_generator(p_right, k - k_mod):
                ps_all = ps_left + ps
                ns_all = ns_left + [k_div - ps[idx] for idx in range(k - k_mod)]
                if check(ps_all, ns_all):
                    yield ps_all, ns_all
        #elif p_left > 0 and p_left < total_left and p_right < total_right and n_left < total_left and n_right < total_right:
        elif p_left > 0 and p_left < total_left:
            print('bbb')
            for ps_left in all_integer_partitioning_generator(p_left, k_mod):
                ns_left = [k_div + 1 - ps_left[idx] for idx in range(k_mod)]
                for ps_right in all_integer_partitioning_generator(p_right, k - k_mod):
                    ns_right = [k_div - ps_right[idx] for idx in range(k - k_mod)]
                    ps_all = ps_left + ps_right
                    ns_all = ns_left + ns_right
                    if check(ps_all, ns_all):
                        yield ps_all, ns_all
        elif p_left == total_left:
            print('ccc')
            ps_left = [k_div + 1] * k_mod
            ns_left = [0] * k_mod
            for ps in all_integer_partitioning_generator(p_right, k - k_mod):
                ps_all = ps_left + ps
                ns_all = ns_left + [k_div - ps[idx] for idx in range(k - k_mod)]
                if check(ps_all, ns_all):
                    yield ps_all, ns_all

            break


In [88]:
def all_kfolds_generator2(p, n, k):
    k_div = (p + n) // k
    k_mod = (p + n) % k

    total_left = k_mod * (k_div + 1)
    total_right = (k - k_mod) * k_div

    max_n_right = min(total_right - (k - k_mod), n - k_mod)
    min_n_left = n - max_n_right
    max_p_left = total_left - min_n_left
    max_p_right = min(total_right - (k - k_mod), p - k_mod)
    min_p_left = p - max_p_right

    for p_left in range(min_p_left, max_p_left+1):
        p_right = p - p_left

        for ps_left in integer_partitioning_generator(p_left, k_mod):
            if any(psl == k_div + 1 for psl in ps_left):
                continue

            ns_left = [k_div + 1 - ps_left[idx] for idx in range(k_mod)]

            for ps_right in integer_partitioning_generator(p_right, k - k_mod):

                if any(psr == k_div for psr in ps_right):
                    continue

                ns_right = [k_div - ps_right[idx] for idx in range(k - k_mod)]

                ps_all = ps_left + ps_right
                ns_all = ns_left + ns_right

                res = sorted(list(zip(ps_all, ns_all, strict=False)), key=lambda x: (x[0], x[1]))

                yield list(zip(*res, strict=False))


In [89]:
[x for x in all_kfolds_generator2(5, 5, 3)]

[[(1, 2, 2), (3, 1, 1)], [(1, 2, 2), (2, 1, 2)], [(1, 1, 3), (2, 2, 1)]]

In [83]:
[tmp for tmp in fold_partitioning_generator(5, 5, 3)]

[([1, 1, 3], [2, 2, 1]), ([1, 2, 2], [3, 1, 1]), ([1, 2, 2], [2, 1, 2])]

In [68]:
[x for x in all_integer_partitioning_generator(2, 4)]

[[0, 0, 0, 2], [0, 0, 1, 1]]

In [4]:
[tuple(x)
        for x in itertools.combinations([1, 2, 2, 3],
                                        3)]

[(1, 2, 2), (1, 2, 3), (1, 2, 3), (2, 2, 3)]

In [33]:
def multiplicity_combinations(values, k):
    for comb in itertools.combinations(list(range(len(values))), k):
        for idx, cdx in enumerate(comb):
            if cdx < len(values) - 1 and (values[cdx] == values[cdx + 1] and (idx == k-1 or comb[idx + 1] != cdx + 1)):
                break
        else:
            yield comb


In [52]:
[x for x in integer_partitioning_generator(2, 0)]

[[]]

In [34]:
import copy

In [37]:
def fpg2(p, n, n_folds, p_non_zero):
    max_items = (p + n) // n_folds
    remainder = (p + n) % n_folds

    for ps in integer_partitioning_generator(p, p_non_zero): # pylint: disable=invalid-name
        ps = [0] * (n_folds - p_non_zero) + ps

        if sum(item > max_items + 1 for item in ps) > 0:
            continue

        n_maxp1 = sum(item == max_items + 1 for item in ps)
        n_max = sum(item == max_items for item in ps)
        if n_maxp1 > remainder or n_maxp1 > 2:
            continue

        if n_maxp1 + n_max > n_folds - 2:
            continue

        if n_maxp1 + n_max > p_non_zero - 2:
            continue

        n_ordinary = len(ps) - sum(item == max_items for item in ps)
        ns = [max_items - p_val + (idx >= n_ordinary) for idx, p_val in enumerate(ps)] # pylint: disable=invalid-name

        for comb in multiplicity_combinations(ps[:n_ordinary], remainder - (len(ps) - n_ordinary)):
            ns_tmp = copy.copy(ns)
            for c in comb:
                ns_tmp[c] += 1
            yield ps, ns_tmp



In [42]:
[(ps, ns) for ps, ns in fpg2(6, 6, 3)]

[([1, 2, 3], [3, 2, 1]), ([2, 2, 2], [2, 2, 2])]

In [25]:
[tuple(x)
        for x in multiplicity_combinations([1, 2, 2, 3],
                                        3)]

[(0, 1, 2), (0, 2, 3), (1, 2, 3)]

In [43]:
[tmp for tmp in fold_partitioning_generator(6, 6, 3)]

[([1, 2, 3], [3, 2, 1]), ([2, 2, 2], [2, 2, 2])]

In [44]:
from sklearn.model_selection import KFold

In [48]:
import numpy as np

X = np.random.random_sample(size=(6, 2))
y = np.array([0, 0, 0, 1, 1, 1])

In [49]:
for train, test in KFold(n_splits=3).split(X=X, y=y):
    print(y[test])

[0 0]
[0 1]
[1 1]
